In [1]:
import pandas as pd
import numpy as np
import itertools
import nltk
from nltk.stem import PorterStemmer

In [2]:
book_df=pd.read_csv("collaborative_book_metadata.csv")

In [3]:
book_df.head()

,Unnamed: 0,book_id,title,image_url,url,num_pages,ratings_count,description,genre,name,book_id_mapping
0,0,5899779,Pride and Prejudice and Zombies Pride and Prej...,https://images.gr-assets.com/books/1320449653m...,https://www.goodreads.com/book/show/5899779-pr...,320,105537,The New York Times Best Seller is now a major ...,"['fantasy, paranormal', 'romance', 'fiction', ...",Jane Austen,808
1,1,872333,Blue Bloods Blue Bloods 1,https://images.gr-assets.com/books/1322281515m...,https://www.goodreads.com/book/show/872333.Blu...,302,117633,"When the Mayflower set sail in 1620, it carrie...","['young-adult', 'fantasy, paranormal', 'romanc...",Melissa de la Cruz,217
2,2,15507958,Me Before You Me Before You 1,https://images.gr-assets.com/books/1357108762m...,https://www.goodreads.com/book/show/15507958-m...,369,609327,Louisa Clark is an ordinary young woman living...,"['romance', 'fiction']",Jojo Moyes,385
3,3,66559,Sharp Objects,https://images.gr-assets.com/books/1423241485m...,https://www.goodreads.com/book/show/66559.Shar...,254,208394,"Fresh from a brief stay at a psych hospital, r...","['mystery, thriller, crime', 'fiction']",Gillian Flynn,192
4,4,7235533,The Way of Kings The Stormlight Archive 1,https://images.gr-assets.com/books/1507307887m...,https://www.goodreads.com/book/show/7235533-th...,1007,151473,"Speak again the ancient oaths,\nLife before de...","['fantasy, paranormal', 'fiction']",Brandon Sanderson,873


In [4]:
book_df.duplicated().sum()

0

In [5]:
book=book_df[["book_id","title","genre","description"]]

In [6]:
book.iloc[0]["genre"]

"['fantasy, paranormal', 'romance', 'fiction', 'history, historical fiction, biography', 'young-adult', 'mystery, thriller, crime']"

In [7]:
import ast #for converting str to list

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i) 
    return L

In [8]:
book['genre'] = book['genre'].apply(convert)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\2443777982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book['genre'] = book['genre'].apply(convert)


In [9]:
book.iloc[0]["genre"]

['fantasy, paranormal',
 'romance',
 'fiction',
 'history, historical fiction, biography',
 'young-adult',
 'mystery, thriller, crime']

In [10]:
def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [11]:
book['genre'] = book['genre'].apply(remove_space)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\207363691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book['genre'] = book['genre'].apply(remove_space)


In [12]:
book["description"]=book["description"].apply(lambda x:x.split())

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\3015881702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book["description"]=book["description"].apply(lambda x:x.split())


In [13]:
book["Tags"]=book["genre"]+book["description"]

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\2701985202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book["Tags"]=book["genre"]+book["description"]


In [14]:
book.head()

,book_id,title,genre,description,Tags
0,5899779,Pride and Prejudice and Zombies Pride and Prej...,"[fantasy,paranormal, romance, fiction, history...","[The, New, York, Times, Best, Seller, is, now,...","[fantasy,paranormal, romance, fiction, history..."
1,872333,Blue Bloods Blue Bloods 1,"[young-adult, fantasy,paranormal, romance, fic...","[When, the, Mayflower, set, sail, in, 1620,, i...","[young-adult, fantasy,paranormal, romance, fic..."
2,15507958,Me Before You Me Before You 1,"[romance, fiction]","[Louisa, Clark, is, an, ordinary, young, woman...","[romance, fiction, Louisa, Clark, is, an, ordi..."
3,66559,Sharp Objects,"[mystery,thriller,crime, fiction]","[Fresh, from, a, brief, stay, at, a, psych, ho...","[mystery,thriller,crime, fiction, Fresh, from,..."
4,7235533,The Way of Kings The Stormlight Archive 1,"[fantasy,paranormal, fiction]","[Speak, again, the, ancient, oaths,, Life, bef...","[fantasy,paranormal, fiction, Speak, again, th..."


In [15]:
new_book_df = book[['book_id','title','Tags']]

In [16]:
new_book_df['Tags'] = new_book_df['Tags'].apply(lambda x: " ".join(x))
new_book_df.head()

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\1579840890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_book_df['Tags'] = new_book_df['Tags'].apply(lambda x: " ".join(x))


,book_id,title,Tags
0,5899779,Pride and Prejudice and Zombies Pride and Prej...,"fantasy,paranormal romance fiction history,his..."
1,872333,Blue Bloods Blue Bloods 1,"young-adult fantasy,paranormal romance fiction..."
2,15507958,Me Before You Me Before You 1,romance fiction Louisa Clark is an ordinary yo...
3,66559,Sharp Objects,"mystery,thriller,crime fiction Fresh from a br..."
4,7235533,The Way of Kings The Stormlight Archive 1,"fantasy,paranormal fiction Speak again the anc..."


In [17]:
new_book_df['Tags'] = new_book_df['Tags'].apply(lambda x:x.lower())

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\860502264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_book_df['Tags'] = new_book_df['Tags'].apply(lambda x:x.lower())


In [18]:
ps = PorterStemmer()

In [19]:
def stems(text):
    T = []
    
    for i in text.split():
        T.append(ps.stem(i))
    
    return " ".join(T)

In [20]:
new_book_df['Tags'] = new_book_df['Tags'].apply(stems)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\1391816524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_book_df['Tags'] = new_book_df['Tags'].apply(stems)


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [22]:
vector = cv.fit_transform(new_book_df['Tags']).toarray()

In [23]:
vector.shape

(96, 3415)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
similarity = cosine_similarity(vector)

In [26]:
 similarity.shape

(96, 96)

In [27]:
def recommend2(book_name):
    book_id=[]
    index = new_book_df[new_book_df['title'] == book_name].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_book_df.iloc[i[0]].title)
recommend2("Blue Bloods Blue Bloods 1")

Pretty Little Liars Pretty Little Liars 1
Clockwork Princess The Infernal Devices 3
From Dead to Worse Sookie Stackhouse 8
The Secret The Secret 1
Dead as a Doornail Sookie Stackhouse 5


In [28]:
recommend2("Sharp Objects")

Doctor Sleep
Harry Potter and the Cursed Child Parts One and Two Harry Potter 8
A Child Called It Dave Pelzer 1
The Nanny Diaries Nanny 1
Angus Thongs and FullFrontal Snogging Confessions of Georgia Nicolson 1


In [29]:
new_music=pd.read_csv("Music Info.csv")

In [30]:
new_music.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [31]:
new_music["tags"].fillna(method ='ffill', inplace = True)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\991213037.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_music["tags"].fillna(method ='ffill', inplace = True)


In [32]:
new_music["tags"]=new_music["tags"].apply(lambda x:x.split(','))

In [33]:
new_music["tags"]

0        [rock,  alternative,  indie,  alternative_rock...
1        [rock,  alternative,  indie,  pop,  alternativ...
2        [rock,  alternative,  alternative_rock,  90s, ...
3        [rock,  alternative,  indie,  alternative_rock...
4        [rock,  alternative,  indie,  alternative_rock...
                               ...                        
50678                       [alternative_rock,  punk_rock]
50679         [rock,  alternative_rock,  japanese,  cover]
50680                  [metal,  metalcore,  post_hardcore]
50681                  [metal,  metalcore,  post_hardcore]
50682                                    [rock,  japanese]
Name: tags, Length: 50683, dtype: object

In [34]:
final_book=book_df[["book_id","title","genre","description"]]

In [35]:
final_book["genre"]=final_book['genre'].apply(convert)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\1379630675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_book["genre"]=final_book['genre'].apply(convert)


In [36]:
tag_table_books = [ token for idx, token in enumerate(set(itertools.chain.from_iterable(final_book["genre"])))]
tag_table_books = pd.DataFrame(tag_table_books)
tag_table_books.columns = ['Tag']
tag_table_books["Tag"]=tag_table_books["Tag"].apply(lambda x:x.lower())


In [37]:
tag_book = tag_table_books['Tag'].values.ravel()
tag_book


array(['children', 'fantasy, paranormal', 'mystery, thriller, crime',
       'poetry', 'fiction', 'romance', 'comics, graphic',
       'history, historical fiction, biography', 'young-adult',
       'non-fiction'], dtype=object)

In [38]:
tag_table_books

,Tag
0,children
1,"fantasy, paranormal"
2,"mystery, thriller, crime"
3,poetry
4,fiction
5,romance
6,"comics, graphic"
7,"history, historical fiction, biography"
8,young-adult
9,non-fiction


In [39]:
music_tags=new_music["tags"]
tag_table_new_music = [ token for idx, token in enumerate(set(itertools.chain.from_iterable(music_tags)))]
tag_table_new_music = pd.DataFrame(tag_table_new_music)
tag_table_new_music.columns = ['Tag']

tag_new_music = tag_table_new_music.values.ravel()
tag_new_music

array(['post_rock', ' house', 'noise', 'beautiful', ' soul',
       ' dark_ambient', ' electronic', ' rnb', 'grindcore', 'british',
       'doom_metal', 'oldies', ' female_vocalists', 'house',
       ' classic_rock', 'male_vocalists', ' metal', ' thrash_metal',
       'indie', 'trip_hop', 'guitar', ' folk', ' nu_metal', 'alternative',
       'soul', 'hip_hop', ' new_wave', ' oldies', 'classical',
       'gothic_metal', 'gothic', ' black_metal', 'pop', 'synthpop', '60s',
       'ska', 'instrumental', ' 60s', ' acoustic', 'progressive_metal',
       ' 90s', '70s', ' trance', 'rap', ' post_rock', 'electronic',
       'polish', ' electro', 'piano', ' dance', 'black_metal',
       ' symphonic_metal', 'experimental', ' russian', ' funk', 'german',
       'rnb', 'idm', ' alternative_rock', ' country', 'new_wave', '00s',
       ' synthpop', 'acoustic', 'nu_metal', ' classical', 'french',
       ' 70s', ' blues', 'swedish', ' techno', 'techno', ' chill',
       ' grunge', ' progressive_rock', '

In [40]:
import nltk
from nltk.corpus import wordnet as wn

In [41]:
book_new_music_map = {} # Creating a dictionary of te mapping
def find_relation(str):
    value=str
    str = str.replace('-', ' ')
    if(len(str.split())==1):
        a = wn.synsets(str)
        if len(a) == 0:
            return 'Null'
        else:
            x = []
            for genre in tag_book:
                m = wn.synsets(genre)
                if len(m)>0:
                   x.append(a[0].path_similarity(m[0]))
            x = np.array(x, dtype = float)
            max_idx = x.argmax()
            return tag_book[max_idx]
    else:
        a = wn.synsets(str.split()[0])
        b = wn.synsets(str.split()[1])
        if len(a) == 0 and len(b) == 0:
            return 'Null'
        elif (len(a) != 0 and len(b) == 0) or (len(a) == 0 and len(b) != 0):
            if len(a) != 0 and len(b) == 0:
                a = wn.synsets(str.split()[0])
            else:
                 a = wn.synsets(str.split()[1])
            if len(a) == 0:
                return 'Null'
            else:
                x = []
                for genre in tag_book:
                    m = wn.synsets(genre)
                    if len(m)>0:
                       x.append(a[0].path_similarity(m[0]))
                x = np.array(x, dtype = float)
                max_idx = x.argmax()
                return tag_book[max_idx]
        else:
            x = []
            y = []
            for genre in tag_book:
                m = wn.synsets(genre)
                if len(m)>0:
                   x.append(a[0].path_similarity(m[0]))
                   y.append(b[0].path_similarity(m[0]))
            x = np.array(x, dtype = float)
            y = np.array(y, dtype = float)
            if (x.max() > y.max()):
                max_idx = x.argmax()
            else:
                max_idx = y.argmax()
                return tag_book[max_idx]
    
tag_table_new_music['relation'] = tag_table_new_music['Tag'].apply(find_relation)

In [42]:
tag_table_new_music

,Tag,relation
0,post_rock,Null
1,house,Null
2,noise,"fantasy, paranormal"
3,beautiful,"fantasy, paranormal"
4,soul,Null
...,...,...
194,dark_ambient,Null
195,hip_hop,Null
196,singer_songwriter,Null
197,new_age,Null


In [43]:
import random as rd

In [44]:
for i in range(len(tag_table_new_music)):
        if tag_table_new_music.iloc[i]["relation"]=="Null" :
            tag_table_new_music.at[i,"relation"]=rd.choice(tag_book)
            print("upadated value" ,tag_table_new_music.iloc[i]["relation"])

upadated value fantasy, paranormal
upadated value children
upadated value poetry
upadated value history, historical fiction, biography
upadated value fantasy, paranormal
upadated value young-adult
upadated value non-fiction
upadated value romance
upadated value mystery, thriller, crime
upadated value history, historical fiction, biography
upadated value mystery, thriller, crime
upadated value fiction
upadated value poetry
upadated value comics, graphic
upadated value non-fiction
upadated value romance
upadated value non-fiction
upadated value children
upadated value poetry
upadated value romance
upadated value mystery, thriller, crime
upadated value young-adult
upadated value young-adult
upadated value non-fiction
upadated value mystery, thriller, crime
upadated value romance
upadated value non-fiction
upadated value non-fiction
upadated value young-adult
upadated value non-fiction
upadated value mystery, thriller, crime
upadated value history, historical fiction, biography
upadated va

In [45]:
tag_table_new_music

,Tag,relation
0,post_rock,"fantasy, paranormal"
1,house,children
2,noise,"fantasy, paranormal"
3,beautiful,"fantasy, paranormal"
4,soul,poetry
...,...,...
194,dark_ambient,romance
195,hip_hop,young-adult
196,singer_songwriter,fiction
197,new_age,children


In [46]:
tag_table_new_music.fillna(method ='bfill', inplace = True)

C:\Users\Mahesh\AppData\Local\Temp\ipykernel_23448\4032516536.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tag_table_new_music.fillna(method ='bfill', inplace = True)


In [47]:
new_music=new_music[['spotify_id', 'name', 'artist','tags']]

In [48]:
new_music.head()

,spotify_id,name,artist,tags
0,09ZQ5TmUG8TSL56n0knqrj,Mr. Brightside,The Killers,"[rock, alternative, indie, alternative_rock..."
1,06UfBBDISthj1ZJAtX4xjj,Wonderwall,Oasis,"[rock, alternative, indie, pop, alternativ..."
2,0keNu0t0tqsWtExGM3nT1D,Come as You Are,Nirvana,"[rock, alternative, alternative_rock, 90s, ..."
3,0ancVQ9wEcHVd0RrGICTE4,Take Me Out,Franz Ferdinand,"[rock, alternative, indie, alternative_rock..."
4,01QoK9DA7VTeTSE3MNzp4I,Creep,Radiohead,"[rock, alternative, indie, alternative_rock..."


In [49]:
new_music_col = list(new_music.columns)
new_music_tags = new_music['tags']
tag_table_music=[[token, idx] for idx, token in enumerate(set(itertools.chain.from_iterable(new_music_tags)))]
tag_table_music = pd.DataFrame(tag_table_music)
tag_table_music.columns = ['Tag', 'Index']

In [50]:
tag_table_music

,Tag,Index
0,post_rock,0
1,house,1
2,noise,2
3,beautiful,3
4,soul,4
...,...,...
194,dark_ambient,194
195,hip_hop,195
196,singer_songwriter,196
197,new_age,197


In [51]:
new_tag_dummy_music = np.zeros([len(new_music), len(tag_table_music)])

In [52]:
for i in range(len(new_music)):
    for j in range(len(tag_table_music)):
        if tag_table_music['Tag'][j] in  new_music.iloc[i]["tags"]:
            new_tag_dummy_music[i, j] = 1
            
new_hot_encoded_music = pd.concat([new_music, pd.DataFrame(new_tag_dummy_music)],axis = 1)
new_music_col.extend([string for string in tag_table_music['Tag']])
new_hot_encoded_music.columns = new_music_col
new_hot_encoded_music.head()

,spotify_id,name,artist,tags,post_rock,house,noise,beautiful,soul,dark_ambient,...,screamo,britpop,pop_rock,singer_songwriter,ambient,dark_ambient,hip_hop,singer_songwriter,new_age,downtempo
0,09ZQ5TmUG8TSL56n0knqrj,Mr. Brightside,The Killers,"[rock, alternative, indie, alternative_rock...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,06UfBBDISthj1ZJAtX4xjj,Wonderwall,Oasis,"[rock, alternative, indie, pop, alternativ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0keNu0t0tqsWtExGM3nT1D,Come as You Are,Nirvana,"[rock, alternative, alternative_rock, 90s, ...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0ancVQ9wEcHVd0RrGICTE4,Take Me Out,Franz Ferdinand,"[rock, alternative, indie, alternative_rock...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01QoK9DA7VTeTSE3MNzp4I,Creep,Radiohead,"[rock, alternative, indie, alternative_rock...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
def lower_case(tag_selected_book):
    list=[]
    for i in tag_selected_book:
        list.append(i.lower())
    return list  

In [54]:
def recommend(book_name):
    book_id=[]
    index = new_book_df[new_book_df['title'] == book_name].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        book_id.append(new_book_df.iloc[i[0]].book_id)
    filtered_dataset = pd.DataFrame([ final_book[final_book.book_id == i].values[0] for i in book_id], columns = ['book_id', 'title','genre',"description"])
    selected_book_tags = filtered_dataset['genre']
    tag_selected_book = [ token for idx, token in enumerate(set(itertools.chain.from_iterable(selected_book_tags)))]
    tag_selected_book=lower_case(tag_selected_book)
    new_tag_selected_music = [tag_table_new_music[tag_table_new_music.relation == i].Tag.values for i in tag_selected_book]
    new_tag_selected_music = [ token for idx, token in enumerate(set(itertools.chain.from_iterable(new_tag_selected_music)))]
    recommended_new_music = []
    counts = np.zeros((new_hot_encoded_music.shape[0]))
    for i in new_tag_selected_music:
        indices = new_hot_encoded_music[new_hot_encoded_music[i] == 1.0].index.tolist()
        for j in indices:
            counts[j] += 1.0
    for i in range(40):
        max_idx = counts.argmax()
        recommended_new_music.append(max_idx)
        counts[max_idx] = 0
    recommended_new_music = new_hot_encoded_music.iloc[recommended_new_music].name.values
    recommended_new_music = np.array(recommended_new_music)
    recommended_new_music = recommended_new_music.ravel()
    return  recommended_new_music

In [55]:
recommend("Sharp Objects")

array(['Benjamin', 'Now or Never', 'Lush Life', 'Hotter Than Hell',
       'Grey Days', 'The Abyss', 'Spun', 'Strain', 'Offering', 'Scrape',
       'Welt', 'The Warden', 'Flatlands', 'Lowlife', 'In The Woods',
       'Simple Death', 'Altar', 'Junction', 'Witness', 'Moses',
       'The Wasteland', 'More And More', 'Point of No Return',
       "I Can't Help You", 'Kill Fix Demo', "Shinin' On",
       "Won't Drag Me Down", 'Candy Andy Jane', 'In Mourning',
       'Het Is Over', 'Shadows/Light', 'Survive', 'Jukebox', 'Suitcases',
       'Blameless', 'Bottomless Pit', 'Vex', 'Brian Boru',
       'Yours Forever', 'That Other Girl'], dtype=object)

In [56]:
recommend("Blue Bloods Blue Bloods 1")

array(['Now or Never', 'Benjamin', 'Lush Life', 'Hotter Than Hell',
       'Grey Days', 'The Abyss', 'Spun', 'Strain', 'Offering',
       'The Warden', 'Scrape', 'Welt', 'Flatlands', 'Lowlife',
       'In The Woods', 'Simple Death', 'Altar', 'Witness',
       'The Wasteland', 'Junction', 'Moses', 'Point of No Return',
       "I Can't Help You", 'Kill Fix Demo', "Shinin' On",
       "Won't Drag Me Down", 'Candy Andy Jane', 'In Mourning',
       'Het Is Over', 'Shadows/Light', 'More And More', 'Survive',
       'Brian Boru', 'Vex', 'That Other Girl', 'Hilferuf', 'Der Brief',
       'Jukebox', 'Suitcases', 'Blameless'], dtype=object)

In [57]:
recommend("Me Before You Me Before You 1")

array(['Now or Never', 'Benjamin', 'Lush Life', 'Hotter Than Hell',
       'Grey Days', 'The Abyss', 'Spun', 'Strain', 'Offering',
       'The Warden', 'Scrape', 'Welt', 'Flatlands', 'Lowlife',
       'In The Woods', 'Simple Death', 'Moses', 'The Wasteland', 'Altar',
       'Junction', 'Witness', 'Point of No Return', "I Can't Help You",
       'Kill Fix Demo', "Shinin' On", "Won't Drag Me Down",
       'Candy Andy Jane', 'In Mourning', 'Het Is Over', 'Shadows/Light',
       'More And More', 'Survive', 'Brian Boru', 'Suitcases', 'Blameless',
       'Vex', 'Der Brief', 'Jukebox', 'Yours Forever', 'That Other Girl'],
      dtype=object)

In [58]:
import pickle

In [59]:
pickle.dump(new_book_df,open('artifacts/book_df.pkl','wb'))
pickle.dump(similarity,open('artifacts/similarity.pkl','wb'))
pickle.dump(final_book,open('artifacts/final_book.pkl','wb'))
pickle.dump(tag_table_new_music,open('artifacts/tag_table_new_music.pkl','wb'))
pickle.dump(new_hot_encoded_music,open('artifacts/new_hot_encoded_music.pkl','wb'))

In [60]:
book_titles=book.title.unique().tolist()

In [61]:
book_titles=pd.DataFrame(book_titles)
book_titles.columns=["titles"]
book_titles.to_json("book_titles_new.json")

In [62]:
bk=book.title.unique().tolist()

In [63]:
results = [{'title': song} for song in bk]
results

[{'title': 'Pride and Prejudice and Zombies Pride and Prejudice and Zombies 1'},
 {'title': 'Blue Bloods Blue Bloods 1'},
 {'title': 'Me Before You Me Before You 1'},
 {'title': 'Sharp Objects'},
 {'title': 'The Way of Kings The Stormlight Archive 1'},
 {'title': 'The Andromeda Strain'},
 {'title': 'A Dance with Dragons A Song of Ice and Fire 5'},
 {'title': 'A Clash of Kings A Song of Ice and Fire 2'},
 {'title': 'The Eye of the World Wheel of Time 1'},
 {'title': 'The Great Hunt Wheel of Time 2'},
 {'title': 'Eleven Minutes'},
 {'title': 'The Ultimate Hitchhikers Guide to the Galaxy'},
 {'title': 'Hamlet Screenplay Introduction And Film Diary'},
 {'title': 'From Dead to Worse Sookie Stackhouse 8'},
 {'title': 'Dead as a Doornail Sookie Stackhouse 5'},
 {'title': 'Living Dead in Dallas Sookie Stackhouse 2'},
 {'title': 'The Girl Who Loved Tom Gordon'},
 {'title': 'Doctor Sleep'},
 {'title': 'Harry Potter Boxset Harry Potter 17'},
 {'title': 'Harry Potter and the Cursed Child Parts One

In [64]:
recommend2("Pretty Little Liars Pretty Little Liars 1")

Blue Bloods Blue Bloods 1
Clockwork Princess The Infernal Devices 3
The Secret The Secret 1
A Wizard of Earthsea Earthsea Cycle 1
The City of Ember Book of Ember 1


In [65]:
recommend2("Clockwork Princess The Infernal Devices 3")

The TellTale Heart and Other Writings
I Am the Messenger
The Son of Neptune The Heroes of Olympus 2
Free Four Tobias Tells the Divergent KnifeThrowing Scene Divergent 15
Maus I A Survivors Tale My Father Bleeds History Maus 1


In [66]:
recommend2("From Dead to Worse Sookie Stackhouse 8")

Dead as a Doornail Sookie Stackhouse 5
Living Dead in Dallas Sookie Stackhouse 2
The Walking Dead Vol 01 Days Gone Bye
Betrayed House of Night 2
Blue Bloods Blue Bloods 1


In [67]:
recommend2("The Secret The Secret 1")

Pretty Little Liars Pretty Little Liars 1
Blue Bloods Blue Bloods 1
Slammed Slammed 1
Persepolis The Story of a Childhood Persepolis 1
The Five People You Meet in Heaven


In [68]:
recommend2("Dead as a Doornail Sookie Stackhouse 5")

From Dead to Worse Sookie Stackhouse 8
Living Dead in Dallas Sookie Stackhouse 2
Fallen Too Far Rosemary Beach 1 Too Far 1
The Walking Dead Vol 01 Days Gone Bye
I Am the Messenger
